##Installation of the libraries

In [ ]:
!pip install -q transformers

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:00


In [ ]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Torch optimization.

##All libraries needed for training

In [ ]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting rid of the sequences which are outside of the threshold (64-128).

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [ ]:
file_path = "/content/AF-O53113-F1-model_v4.cif"
file_model = "AF-O53113-F1-model_v4.cif"
pdbl = PDBList()
#pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

MKKLIFVVLLLAVISYTYPSNLDYKYTSSIPIEFVKLSEIKNVDELNRLLNSNALVIFCLDSNNINKDILYHLGIKRYNPSEIPDYGNYSYVSINGVIVVYPKYSVYEENGALIYNPPKEENYSKYEFVRPYKIKVPNVSKIPDYGGYVLIENSTFILYPKKYIIRGDEGGIYYIPPKNSSDNYYYKYSYNLPEKCIPDYYDYVFIDNNGIFIIYPKKYVVRSNDGVLVFSPPVDAKEAPIYKIDYKKIDEGVYEIKKNKLLFLYPTTLNNKSTLDIIGEYIAKNGGVFAYINKVPPYYKHMLATGVAIDKVIPDESGSYAIDVAGRKIKVDVLDDEIINNKLKCIKALKALGINVSYIVTGSEGIDIVEKSNVDTDELEDLYNYYWFKKWWQNYTHLYFNPSQLKNIEFNGFEDILALSYYPLIYVDKAPETFRNDPIGGYYPKVISYKGTKNYGYWEEGAKSENVYYHLDEGEPYWDGKANEPSKWYYEGQPVSMENDSEMWNRYEYFNHWFVKNYAYALANGCDGLFLESSDKNLIDAIFGNDNENLSWKLDIKNKVDYVVIPGNKGFEVINGIPVIRIPSPLKEVYGANVINTLYIPPKDEDFGIYISDIRNYNTNLVVELKENGTDIVSFKDLADWLNKYYRNNIFYNGTAIKIWDNRGIKITIFKKNFGMGNYTFEEFDKEHCKYVIVNPPKIIPLK


Calculating the angles for the given sequence

In [ ]:
'''
atom_coords = []

for residue in chain_A:

    if 'N' in residue and 'CA' in residue and 'C' in residue:

        n_coord = residue['N'].get_coord()
        ca_coord = residue['CA'].get_coord()
        c_coord = residue['C'].get_coord()

        atom_coords.append(n_coord)
        atom_coords.append(ca_coord)
        atom_coords.append(c_coord)

        #apparently this is also an extra atom that sometimes exist called Cb so we need to be careful but that will complicate things so for now im ignoring it
        #if 'CB' in residue:
        #    cb_coord = residue['CB'].get_coord()
        #   atom_coords.append(cb_coord)

atom_coords = np.array(atom_coords)
# Now atom_coords contains the XYZ coordinates of the selected atoms
print(atom_coords)
'''

"\natom_coords = []\n\nfor residue in chain_A:\n\n    if 'N' in residue and 'CA' in residue and 'C' in residue:\n\n        n_coord = residue['N'].get_coord()\n        ca_coord = residue['CA'].get_coord()\n        c_coord = residue['C'].get_coord()\n\n        atom_coords.append(n_coord)\n        atom_coords.append(ca_coord)\n        atom_coords.append(c_coord)\n        \n        #apparently this is also an extra atom that sometimes exist called Cb so we need to be careful but that will complicate things so for now im ignoring it\n        #if 'CB' in residue:\n        #    cb_coord = residue['CB'].get_coord()\n        #   atom_coords.append(cb_coord)\n\natom_coords = np.array(atom_coords)\n# Now atom_coords contains the XYZ coordinates of the selected atoms\nprint(atom_coords)\n"

In [ ]:
import numpy as np
'''

#use this code to update the predicted matrix into pdb file
#should have dimensions of atom_coords, im printing it so you can check the dimensions
print(np.shape(atom_coords))

# Initialize a counter for your new coordinates
coord_idx = 0
residue_cnt = 0

# Loop through each residue and update the coordinates of the N, CA, and C atoms
for model in structure:
    for chain in model:
        for residue in chain:
            residue_cnt += 1
            if 'N' in residue and 'CA' in residue and 'C' in residue:
                # Update N atom
                residue['N'].set_coord(atom_coords[coord_idx])
                coord_idx += 1
                # Update CA atom
                residue['CA'].set_coord(atom_coords[coord_idx])
                coord_idx += 1
                # Update C atom
                residue['C'].set_coord(atom_coords[coord_idx])
                coord_idx += 1

print(residue_cnt)
#notice how there are
#structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file
io.save('xyzcoords.pdb',  preserve_atom_numbering=True)
'''

"\n\n#use this code to update the predicted matrix into pdb file\n#should have dimensions of atom_coords, im printing it so you can check the dimensions\nprint(np.shape(atom_coords))\n\n# Initialize a counter for your new coordinates\ncoord_idx = 0\nresidue_cnt = 0\n\n# Loop through each residue and update the coordinates of the N, CA, and C atoms\nfor model in structure:\n    for chain in model:\n        for residue in chain:\n            residue_cnt += 1\n            if 'N' in residue and 'CA' in residue and 'C' in residue:\n                # Update N atom\n                residue['N'].set_coord(atom_coords[coord_idx])\n                coord_idx += 1\n                # Update CA atom\n                residue['CA'].set_coord(atom_coords[coord_idx])\n                coord_idx += 1\n                # Update C atom\n                residue['C'].set_coord(atom_coords[coord_idx])\n                coord_idx += 1\n\nprint(residue_cnt)\n#notice how there are\n#structure.internal_to_atom_coord

Putting angles in a matrix.

delete this later

In [ ]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []


for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


#structure.internal_to_atom_coordinates(verbose = True)
#io = PDBIO() #this is to write a pdb file again
#io.set_structure(structure)#set structure, the structure you wan tin the pdb file
#io.save('AF-O53113-F1-model_v4_coded.pdb',  preserve_atom_numbering=True)

#structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)

angles = np.vstack((psi,phi))

np.shape(angles)

'''
cnt = 0


for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        d[key].angle = angles[0,cnt]


    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
       d[key].angle = angles[1,cnt]
       cnt += 1




structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('input .pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering
'''

"\ncnt = 0\n\n\nfor key in d:\n    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':\n        d[key].angle = angles[0,cnt]\n\n        \n    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':\n       d[key].angle = angles[1,cnt] \n       cnt += 1\n\n\n\n\nstructure.internal_to_atom_coordinates(verbose = True)\nio = PDBIO() #this is to write a pdb file again\nio.set_structure(structure)#set structure, the structure you want in the pdb file\nio.save('input .pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering\n"

Changing sequence for to be used in the Prot-Bert embedding.

In [ ]:
seq_example =  ' '.join(sequence)
seq_example

'M K K L I F V V L L L A V I S Y T Y P S N L D Y K Y T S S I P I E F V K L S E I K N V D E L N R L L N S N A L V I F C L D S N N I N K D I L Y H L G I K R Y N P S E I P D Y G N Y S Y V S I N G V I V V Y P K Y S V Y E E N G A L I Y N P P K E E N Y S K Y E F V R P Y K I K V P N V S K I P D Y G G Y V L I E N S T F I L Y P K K Y I I R G D E G G I Y Y I P P K N S S D N Y Y Y K Y S Y N L P E K C I P D Y Y D Y V F I D N N G I F I I Y P K K Y V V R S N D G V L V F S P P V D A K E A P I Y K I D Y K K I D E G V Y E I K K N K L L F L Y P T T L N N K S T L D I I G E Y I A K N G G V F A Y I N K V P P Y Y K H M L A T G V A I D K V I P D E S G S Y A I D V A G R K I K V D V L D D E I I N N K L K C I K A L K A L G I N V S Y I V T G S E G I D I V E K S N V D T D E L E D L Y N Y Y W F K K W W Q N Y T H L Y F N P S Q L K N I E F N G F E D I L A L S Y Y P L I Y V D K A P E T F R N D P I G G Y Y P K V I S Y K G T K N Y G Y W E E G A K S E N V Y Y H L D E G E P Y W D G K A N E P S K W Y Y E G Q P V S M E N D

##Embedding space creation (using Prot-Bert)

In [ ]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

Getting the dimensions of the embedding space.

In [ ]:
'''
N , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]
embedding_prot_bert = output.last_hidden_state
'''

'\nN , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]\nembedding_prot_bert = output.last_hidden_state\n'

In [ ]:
'''
print(N)
print(D)
'''

'\nprint(N)\nprint(D)\n'

##Single-head self unmasked attention layer

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)
        return attention_output

##Encoder with attention and 2 layer FFNN

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.ReLU(),
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.ReLU(),
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        #output = self.layer_norm2(ff_output)
        return ff_output


In [ ]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.05)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")




In [ ]:
#with using the mse architecture idea
class TransformerTrainer2:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.05)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)
        self.empty_tensor = torch.from_numpy(np.zeros(np.shape(angles.T)))

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss = self.criterion(MeanAngleErrorLoss(predictions,self.angles_tensor).squeeze(), self.empty_tensor)

            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")



In [ ]:
class MeanAngleErrorLoss(nn.Module):
  def __init__(self, predictions, angles):
        super(MeanAngleErrorLoss, self).__init__()
  def forward(self, predictions, angles):
    '''
    predictions_np = predictions.detach().cpu().numpy()#turning it to numpy arrays
    angles_np = angles.detach().cpu().numpy()

    angular_diff = np.abs(predictions_np- angles_np)
    alt_diff = 2*np.pi - angular_diff

    print(self.shape)
    error_phi = []
    error_psi = []
    for i in range(len(predictions_np[:,0])):
       angular_diff = np.abs(predictions_np[i,0]- angles_np[i,0])
       alt_diff = 360 - angular_diff
       error_phi.append(min(angular_diff, alt_diff))
       angular_diff = np.abs(predictions_np[i,1]- angles_np[i,1])
       alt_diff = 360 - angular_diff
       error_psi.append(min(angular_diff, alt_diff))

    error_phi = np.array(error_phi)
    error_psi = np.array(error_psi)

    error_combined = np.column_stack((error_phi, error_psi))
    return torch.from_numpy(error_combined)
    '''
    angular_diff_phi = torch.abs(predictions[:, 0] - angles[:, 0])
    alt_diff_phi = 2*torch.pi - angular_diff_phi
    error_phi = torch.min(angular_diff_phi, alt_diff_phi)

    angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
    alt_diff_psi = 2*torch.pi - angular_diff_psi
    error_psi = torch.min(angular_diff_psi, alt_diff_psi)

    error_combined = torch.stack((error_phi, error_psi), dim=1)
    return error_combined





In [ ]:
angles.T[:, 0].shape
angles

array([[-3.0019474 ,  3.088406  ,  3.1186235 , ..., -3.1220431 ,
         3.0670881 ,  0.        ],
       [-1.225698  , -0.8935808 , -0.5575172 , ...,  2.4149952 ,
         0.32593784,  0.        ]], dtype=float32)

In [ ]:
class TransformerTrainer2:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.05)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            loss.backward(retain_graph=True)
            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

#with mea scalar output
class MeanAngleErrorLoss(nn.Module):
    def __init__(self, predictions, angles):
        super(MeanAngleErrorLoss, self).__init__()

    def forward(self, predictions, angles):
        angular_diff_phi = torch.abs(predictions[:, 0] - angles[:, 0])
        alt_diff_phi = 2*torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2*torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)

        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #mse = torch.mean(error_combined ** 2)
        return error_combined





In [ ]:
#with  using mean angle error loss directly
class TransformerTrainer2:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            loss.backward(retain_graph=True)
            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss, self).__init__()

    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] - angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)
        #print(predictions)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)


        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return mse

In [ ]:
#with  using mean angle error loss with mse loss
class TransformerTrainer3:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T).float()
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)
        self.empty_tensor = torch.zeros_like(self.angles_tensor)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            mean_angle_error = MeanAngleErrorLoss2()(predictions.squeeze(), self.angles_tensor)
            loss = self.criterion(mean_angle_error, self.empty_tensor)


            loss.backward(retain_graph=True)
            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss2(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss2, self).__init__()


    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] -angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)

        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        #mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return error_combined

In [ ]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 20

'''
model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
#model.forward(embeddings)
#criterion = MeanAngleErrorLoss()
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model
'''

'\nmodel = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)\n#model.forward(embeddings)\n#criterion = MeanAngleErrorLoss()\ncriterion = nn.MSELoss()\n#criterion = nn.L1Loss()\ntrainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles)\ntrainer.train()\nmodel\n'

In [ ]:

model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
#model.forward(embeddings)
#criterion = MeanAngleErrorLoss()
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
trainer = TransformerTrainer3(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model

Epoch 1/20, Loss: 2.2573182582855225
Epoch 2/20, Loss: 2.2323081493377686
Epoch 3/20, Loss: 2.194566011428833
Epoch 4/20, Loss: 2.148908853530884
Epoch 5/20, Loss: 2.0980870723724365
Epoch 6/20, Loss: 2.043680429458618
Epoch 7/20, Loss: 1.9868818521499634
Epoch 8/20, Loss: 1.927472472190857
Epoch 9/20, Loss: 1.8650704622268677
Epoch 10/20, Loss: 1.7980220317840576
Epoch 11/20, Loss: 1.7259459495544434
Epoch 12/20, Loss: 1.647971272468567
Epoch 13/20, Loss: 1.5624860525131226
Epoch 14/20, Loss: 1.4680578708648682
Epoch 15/20, Loss: 1.3635672330856323
Epoch 16/20, Loss: 1.2471768856048584
Epoch 17/20, Loss: 1.1168537139892578
Epoch 18/20, Loss: 0.9696646332740784
Epoch 19/20, Loss: 0.8181232213973999
Epoch 20/20, Loss: 0.7960692048072815


TransformerModel(
  (self_attention): SelfAttention()
  (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
  (feed_forward): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)

In [ ]:

angles_tensor = torch.from_numpy(angles.T)

predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [ ]:
pred = predicted_angles*(180/np.pi)
pred


tensor([[[-181.3975,   67.4064],
         [-188.3036,   69.5110],
         [-167.0988,   63.8460],
         ...,
         [-186.5330,   69.1219],
         [-175.2244,   71.4057],
         [-186.2243,   69.2317]]], grad_fn=<MulBackward0>)

In [ ]:
angles_tensor = angles_tensor*(180/np.pi)

In [ ]:
np.shape(pred)


torch.Size([1, 703, 2])

In [ ]:
angles_tensor

tensor([[-171.9989,  -70.2273],
        [ 176.9526,  -51.1984],
        [ 178.6840,  -31.9434],
        ...,
        [-178.8799,  138.3690],
        [ 175.7312,   18.6749],
        [   0.0000,    0.0000]])

In [ ]:
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = pred[0,cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = pred[0,cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('pred .pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering



AF-O53113-F1-model_v4.cif A coordinates for 5786 dihedra updated in 2112 iterations


In [ ]:
zstructure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[0,cnt].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[1,cnt].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('AF-O06917-F1-model_v4_amnglechanged .pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering


NameError: name 'zstructure' is not defined